<a href="https://colab.research.google.com/github/saishr97/Google-Colab-RSS/blob/main/Advanced_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Case Summary: Advanced E-Commerce Data Wrangling** 🛒

### Business Context:

An e-commerce company wants to analyze customer behavior and product sales trends to improve retention and optimize product offerings. They have a dataset of sales transactions and need insights into:

  1.   Customer retention rates - How many customers return within 30 days?

  2. Revenue trends - How does revenue fluctuate daily?

  3. Product category performance - Which categories are growing the fastest?

  4. Best-selling products - What are the top revenue-generating products in each category?

  5. High-value customers - Who are the biggest spenders, and how often do they order?
  

### **Key Challenges**:
Time-sensitive analysis (rolling revenue, retention tracking)
Aggregations over multiple dimensions (customer, category, product)
Comparisons across time periods (month-over-month growth)
Customer segmentation (dividing customers into quartiles based on spending)

###**Expected Outcomes**:
Identify repeat customers and improve retention strategies.
Spot seasonal trends in sales with a 7-day rolling revenue analysis.
Understand which product categories are growing or declining.
Optimize inventory by knowing top-selling products per category.
Categorize customers into spending tiers and tailor marketing accordingly.

###**Business Impact**:
These insights can help boost revenue, improve customer loyalty, and optimize product offerings, leading to higher profitability and better business decisions.

In [2]:
import pandas as pd
import numpy as np

# Sample dataset
data = {
    "order_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    "customer_id": [101, 102, 101, 103, 104, 105, 101, 102, 103, 101, 102, 105, 106, 107, 108],
    "order_date": [
        "2024-01-01", "2024-01-02", "2024-01-15", "2024-01-20", "2024-02-01",
        "2024-02-05", "2024-02-10", "2024-02-12", "2024-02-18", "2024-02-20",
        "2024-02-28", "2024-03-05", "2024-03-10", "2024-03-15", "2024-03-20"
    ],
    "product_category": ["Electronics", "Clothing", "Electronics", "Books", "Books",
                         "Clothing", "Electronics", "Books", "Clothing", "Electronics",
                         "Books", "Electronics", "Clothing", "Books", "Electronics"],
    "product_id": [201, 301, 201, 401, 401, 301, 201, 401, 301, 201, 401, 201, 301, 401, 201],
    "quantity": [1, 2, 1, 3, 2, 1, 4, 1, 2, 3, 5, 2, 3, 1, 4],
    "price": [500, 50, 500, 20, 20, 50, 500, 20, 50, 500, 20, 500, 50, 20, 500]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Convert 'order_date' to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Calculate total revenue per order
df["total_revenue"] = df["quantity"] * df["price"]

# Display DataFrame
df


,order_id,customer_id,order_date,product_category,product_id,quantity,price,total_revenue
0,1,101,2024-01-01,Electronics,201,1,500,500
1,2,102,2024-01-02,Clothing,301,2,50,100
2,3,101,2024-01-15,Electronics,201,1,500,500
3,4,103,2024-01-20,Books,401,3,20,60
4,5,104,2024-02-01,Books,401,2,20,40
5,6,105,2024-02-05,Clothing,301,1,50,50
6,7,101,2024-02-10,Electronics,201,4,500,2000
7,8,102,2024-02-12,Books,401,1,20,20
8,9,103,2024-02-18,Clothing,301,2,50,100
9,10,101,2024-02-20,Electronics,201,3,500,1500


In [4]:
# 1. Customer Retention Analysis: Percentage of customers with a repeat purchase within 30 days
df_sorted = df.sort_values(by=['customer_id', 'order_date'])
df_sorted['next_purchase'] = df_sorted.groupby('customer_id')['order_date'].shift(-1)
df_sorted['days_between'] = (df_sorted['next_purchase'] - df_sorted['order_date']).dt.days
repeat_customers = df_sorted[df_sorted['days_between'] <= 30]['customer_id'].nunique()
total_customers = df_sorted['customer_id'].nunique()
retention_rate = repeat_customers / total_customers * 100

print(f"Percentage of customers with a repeat purchase within 30 days: {retention_rate:.2f}%")

Percentage of customers with a repeat purchase within 30 days: 50.00%


In [5]:
# 2. Rolling Revenue Calculation (7-day window)
df_daily_revenue = df.groupby('order_date')['total_revenue'].sum().reset_index()
df_daily_revenue["7_day_avg_revenue"] = df_daily_revenue["total_revenue"].rolling(window=7, min_periods=1).mean()

print(df_daily_revenue)


   order_date  total_revenue  7_day_avg_revenue
0  2024-01-01            500         500.000000
1  2024-01-02            100         300.000000
2  2024-01-15            500         366.666667
3  2024-01-20             60         290.000000
4  2024-02-01             40         240.000000
5  2024-02-05             50         208.333333
6  2024-02-10           2000         464.285714
7  2024-02-12             20         395.714286
8  2024-02-18            100         395.714286
9  2024-02-20           1500         538.571429
10 2024-02-28            100         544.285714
11 2024-03-05           1000         681.428571
12 2024-03-10            150         695.714286
13 2024-03-15             20         412.857143
14 2024-03-20           2000         695.714286


In [9]:
# 3. Category Growth Rate (Month-over-Month)
df['year_month'] = df['order_date'].dt.to_period('M')
category_monthly_revenue = df.groupby(['year_month', 'product_category'])['total_revenue'].sum().reset_index()
category_monthly_revenue['prev_month_revenue'] = category_monthly_revenue.groupby('product_category')['total_revenue'].shift(1)
category_monthly_revenue['growth_rate'] = ((category_monthly_revenue['total_revenue'] - category_monthly_revenue['prev_month_revenue']) /category_monthly_revenue['prev_month_revenue']) * 100
category_monthly_revenue.sort_values(by=['product_category', 'year_month'], inplace=True)

print(category_monthly_revenue)



  year_month product_category  total_revenue  prev_month_revenue  growth_rate
0    2024-01            Books             60                 NaN          NaN
3    2024-02            Books            160                60.0   166.666667
6    2024-03            Books             20               160.0   -87.500000
1    2024-01         Clothing            100                 NaN          NaN
4    2024-02         Clothing            150               100.0    50.000000
7    2024-03         Clothing            150               150.0     0.000000
2    2024-01      Electronics           1000                 NaN          NaN
5    2024-02      Electronics           3500              1000.0   250.000000
8    2024-03      Electronics           3000              3500.0   -14.285714


In [10]:
# 4. Best-Selling Product Per Category

top_products = df.groupby(['product_category', 'product_id'])['total_revenue'].sum().reset_index()
best_sellers = top_products.loc[top_products.groupby("product_category")["total_revenue"].idxmax()]

print(best_sellers)


  product_category  product_id  total_revenue
0            Books         401            240
1         Clothing         301            400
2      Electronics         201           7500


In [44]:

# 5. High-Value Customer Segmentation
customer_spending = df.groupby('customer_id', as_index=False)['total_revenue'].sum()
customer_spending['quartile'] = pd.qcut(
    customer_spending['total_revenue'],
    4,
    labels=['low', 'medium', 'high', 'VIP']
)

orders_per_quartile = df.groupby('customer_id')['order_id'].count().reset_index()
orders_per_quartile = orders_per_quartile.merge(customer_spending, on='customer_id')
orders_summary = orders_per_quartile.groupby('quartile')['order_id'].mean()

print(orders_summary)



quartile
low       1.0
medium    1.5
high      2.5
VIP       2.5
Name: order_id, dtype: float64


<ipython-input-44-fb761c2bc42a>:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  orders_summary = orders_per_quartile.groupby('quartile')['order_id'].mean()
